In [1]:
from scipy.optimize import minimize
import numpy as np
from scipy import stats

In [2]:
def f(x):
    # Constraint: the mean of the data is approximately 2.3 ± 0.5.
    # Constraint: the variance of the data is approximately 2.75 ± 1.
    mu = x[0]
    nu = x[1]
    alpha = x[2]
    beta = x[3]
    x_mean = mu
    x_sd = np.sqrt(beta/(alpha-1)/nu)
    sigma2_mean = beta/(alpha-1)
    sigma2_sd = beta/(alpha-1)/np.sqrt(alpha-2)
    return (x_mean - 2.3)**2 + (x_sd - 0.5)**2 + (sigma2_mean - 2.75)**2 + (sigma2_sd - 1)**2

In [3]:
x_initial = [2.3, 0.1, 3, 5]
result = minimize(f, x_initial)
x_final = result.x

In [4]:
mu_0 = x_final[0]
nu_0 = x_final[1]
alpha_0 = x_final[2]
beta_0 = x_final[3]
print(x_final)

[ 2.29999994 11.00000285  9.56250049 23.54687626]


In [5]:
'''
Function definitions for the normal-inverse-gamma distribution. The parameters
of the distribution, namely mu, lambda / nu, alpha, beta, are as defined here:

  https://en.wikipedia.org/wiki/Normal-inverse-gamma_distribution

Note that we use the symbol nu (ν) rather than lambda (λ) for the third parameter.
This is to match the notation used in the conjugate priors table on Wikipedia:

  https://en.wikipedia.org/wiki/Conjugate_prior#Table_of_conjugate_distributions
'''

def norminvgamma_pdf(x, sigma2, mu, nu, alpha, beta):
    '''
    The probability density function of the normal-inverse-gamma distribution at
    x (mean) and sigma2 (variance).
    '''
    return (
        stats.norm.pdf(x, loc=mu, scale=np.sqrt(sigma2 / nu)) *
        stats.invgamma.pdf(sigma2, a=alpha, scale=beta))

def norminvgamma_rvs(mu, nu, alpha, beta, size=1):
    '''
    Generate n samples from the normal-inverse-gamma distribution. This function
    returns a (size x 2) matrix where each row contains a sample, (x, sigma2).
    '''
    sigma2 = stats.invgamma.rvs(a=alpha, scale=beta, size=size)  # Sample sigma^2 from the inverse-gamma
    x = stats.norm.rvs(loc=mu, scale=np.sqrt(sigma2 / nu), size=size)  # Sample x from the normal
    return np.vstack((x, sigma2)).transpose()

In [6]:
num_samples = 1000000
samples = norminvgamma_rvs(mu_0, nu_0, alpha_0, beta_0, size=num_samples)
print(samples[:,0].mean(), samples[:,0].std())
print(samples[:,1].mean(), samples[:,1].std())

2.3005117082163222 0.5000650536590753
2.7508227929735427 1.000504594433007
